In [1]:
import pandas as pd
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

# check if GLPK is installed. If not, install.
if not (shutil.which("glpsol") or os.path.isfile("glpsol")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge glpk
        except:
            pass
assert(shutil.which("glpsol") or os.path.isfile("glpsol"))

## Separación no total

In [13]:
# Importar pyomo
from pyomo.environ import *

# Crear un modelo concreto
model = ConcreteModel(name = "NON SHARP SEPARATION ")

# SETS -------------------------------------------------------------------------
# Components (I)
model.I = Set(initialize = ['A', 'B', 'C'], ordered = True, doc = 'components')

# Conventional distillation columns (C)
model.C = Set(initialize = ['c1', 'c2', 'c3', 'c4', 'c5'], ordered = True, doc = 'distillation columns')
model.C0 = Set(initialize = ['c1', 'c2', 'c3'], ordered = True, doc = 'distillation columns')

# ALIAS
model.II = SetOf(model.I)

# PARAMETROS -------------------------------------------------------------------
# Composición del alimento {kmol/h}
model.F0 = Param(model.I, initialize = {'A': 20, 'B': 30, 'C': 50})

# > Coste fijo de las columnas
model.FC = Param(model.C, initialize = {'c1': 100, 'c2': 100, 'c3': 100, 'c4': 75, 'c5': 30})

# > Coste variable de las columnas
model.VC = Param(model.C ,initialize = {'c1': 20, 'c2': 10, 'c3': 15, 'c4': 8, 'c5': 5})

# Column Component fraction in distillate
model.x = Param(model.C, model.I, 
                initialize = { ('c1', 'A'): 0.98, ('c1', 'B'): 0.02, ('c1', 'C'): 0.00,
                                ('c2', 'A'): 0.98, ('c2', 'B'): 0.50, ('c2', 'C'): 0.02,
                                ('c3', 'A'): 0.98, ('c3', 'B'): 0.98, ('c3', 'C'): 0.02,
                                ('c4', 'A'): 1.00, ('c4', 'B'): 0.98, ('c4', 'C'): 0.02,
                                ('c5', 'A'): 0.98, ('c5', 'B'): 0.02, ('c5', 'C'): 0.00,
                                },
)


# VARIABLES --------------------------------------------------------------------
# > Variables continuas
model.col_cost = Var(model.C, domain = NonNegativeReals, doc = 'distillation column c cost')

model.F = Var(model.C, model.I, domain = NonNegativeReals, doc = 'Feed stream to column c and component i')
model.D = Var(model.C, model.I, domain = NonNegativeReals, doc = 'Distillate product of column c and component i')
model.B = Var(model.C, model.I, domain = NonNegativeReals, doc = 'Bottoms product of column c and component i')

model.S1  = Var(model.I, domain = NonNegativeReals, doc = 'Feed stream bypass to node A')
model.S2  = Var(model.I, domain = NonNegativeReals, doc = 'Feed stream bypass to node B')
model.S3  = Var(model.I, domain = NonNegativeReals, doc = 'Feed stream bypass to node C')

model.ProdA = Var(model.I, domain = NonNegativeReals, doc = 'Product stream rich in product A')
model.ProdB = Var(model.I, domain = NonNegativeReals, doc = 'Product stream rich in product B')
model.ProdC = Var(model.I, domain = NonNegativeReals, doc = 'Product stream rich in product C')

# > Variables Binarias
model.y = Var(model.C, domain = Binary, doc = '1 if column c is selected. 0 otherwise')

# MODEL CONSTRAINTS EQUATIONS --------------------------------------------------
# > Restricciones globales
#   - Balance materia en nodo de alimentación
def MB_feed_node_rule(m, i):
    return m.F0[i] == m.S1[i] + m.S2[i] + m.S3[i] + m.F['c1', i] + m.F['c2', i] + m.F['c3', i]
model.MB_feed_node = Constraint(model.I, rule = MB_feed_node_rule)

#   - Balance de materia en el nodo de C4
def MB_feed_column_c4_rule(m, i):
    return m.F['c4', i] == m.B['c1', i] + m.B['c2', i]
model.MB_feed_column_c4 = Constraint(model.I, rule = MB_feed_column_c4_rule)

#   - Balance de materia en el nodo de C5
def MB_feed_column_c5_rule(m, i):
    return m.F['c5', i] == m.D['c2', i] + m.D['c3', i]
model.MB_feed_column_c5 = Constraint(model.I, rule = MB_feed_column_c5_rule)

#   - Balance de materia en el nodo de producto A
def MB_node_A_rule(m, i):
    return m.ProdA[i] == m.S1[i] + m.D['c1', i] + m.D['c5', i]
model.MB_node_A = Constraint(model.I, rule = MB_node_A_rule)

#   - Balance de materia en el nodo de producto B
def MB_node_B_rule(m, i):
    return m.ProdB[i] == m.S2[i] + m.D['c4', i] + m.B['c5', i]
model.MB_node_B = Constraint(model.I, rule = MB_node_B_rule)

#   - Balance de materia en el nodo de producto C
def MB_node_C_rule(m, i):
    return m.ProdC[i] == m.S3[i] + m.B['c3', i] + m.B['c4', i]
model.MB_node_C = Constraint(model.I, rule = MB_node_C_rule)

#   - Igualdad de composición en los splitter
# (stream S1)
def splitter_S1_rule(m, i):
    return sum(m.S1[ii] for ii in m.II) * F0[i] == sum(m.F0[ii] for ii in m.II) * m.S1[i]
model.splitter_S1 = Constraint(model.I, rule = splitter_S1_rule)

# (stream S2)
def splitter_S2_rule(m, i):
    return sum(m.S2[ii] for ii in m.II) * m.F0[i] == sum(m.F0[ii] for ii in m.II) * m.S2[i]
model.splitter_S2 = Constraint(model.I, rule = splitter_S2_rule)

# (stream S3)
def splitter_S3_rule(m, i):
    return sum(m.S3[ii] for ii in m.II) * m.F0[i] == sum(m.F0[ii] for ii in m.II) * m.S3[i]
model.splitter_S3 = Constraint(model.I, rule = splitter_S3_rule)

# (stream F for c1, c2 and c3)
def splitter_F_rule(m, c, i):
    if (c in ['c1', 'c2', 'c3']):
        return sum(m.F[c, ii] for ii in m.II) * m.F0[i] == sum(m.F0[ii] for ii in m.II) * m.F[c, i]
model.splitter_F = Constraint(model.C0, model.I, rule = splitter_F_rule)

#   - Balance de materia en las columnas
def MB_distillation_column_rule(m, c, i):
    return m.F[c, i] == m.D[c, i] + m.B[c, i]
model.MB_distillation_column = Constraint(model.C, model.I, rule = MB_distillation_column_rule)

#   - Corriente de destilado de las columnas de destilación
def distillate_product_rule(m, c, i):
    return m.D[c, i] == m.F[c, i] * m.x[c, i]
model.distillate_product = Constraint(model.C, model.I, rule = distillate_product_rule)


# > Disyunciones (Convex Hull reformulation)
def column_cost_rule(m, c):
    return m.col_cost[c] == m.FC[c] * m.y[c] + m.VC[c] * sum( m.D[c, i] for i in m.I )
model.column_cost = Constraint(model.C, rule = column_cost_rule)

def column_feed_ub_rule(m, c, i):
    return m.F[c,i] <= m.F0[i] * m.y[c]
model.column_feed_ub = Constraint(model.C, model.I, rule = column_feed_ub_rule)


# > Proposiciones lógicas
# Y1 V Y2 V Y3
def logic_proposition_1_rule(m):
    return sum(m.y[c] for c in m.C if c in ['c1', 'c2', 'c3']) == 1
model.logic_proposition_1 = Constraint(rule = logic_proposition_1_rule)

# Y1 ==> Y4
def logic_proposition_2_rule(m):
    return m.y['c1'] - m.y['c4'] <= 0
model.logic_proposition_2 = Constraint(rule = logic_proposition_2_rule)

# Y1 ==> ¬ Y5
def logic_proposition_3_rule(m):
    return  m.y['c1'] + m.y['c5']  <= 1
model.logic_proposition_3 = Constraint(rule = logic_proposition_3_rule)


# FUNCIÓN OBJETIVO  ----------------------------------------------------------
def obj_rule(m):
   return sum( m.col_cost[c] for c in m.C )
model.obj_rule = Objective (rule = obj_rule, sense = minimize )


# FIJAR LÍMITES A LAS VARIABLES -----------------------------------------------------------------
# Product Stream A specifications
model.ProdA['A'].setlb(0.80 * F0['A'])
model.ProdA['B'].setub(0.15 * F0['B'])
model.ProdA['C'].setub(0.05 * F0['C'])

# Product Stream B specifications
model.ProdB['A'].setub(0.10 * F0['A'])
model.ProdB['B'].setlb(0.80 * F0['B'])
model.ProdB['C'].setub(0.10 * F0['C'])

# Product Stream B specifications
model.ProdC['A'].setub(0.05 * F0['A'])
model.ProdC['B'].setub(0.10 * F0['B'])
model.ProdC['C'].setlb(0.80 * F0['C'])

# SOLVER CALL ------------------------------------------------------------------
# > Pyomo solver library
SolverFactory("glpk").solve(model, tee = True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\Isabela\AppData\Local\Temp\tmpdg5prpl5.glpk.raw --wglp C:\Users\Isabela\AppData\Local\Temp\tmpgyga0dd7.glpk.glp
 --cpxlp C:\Users\Isabela\AppData\Local\Temp\tmpah4b5lyg.pyomo.lp
Reading problem data from 'C:\Users\Isabela\AppData\Local\Temp\tmpah4b5lyg.pyomo.lp'...
C:\Users\Isabela\AppData\Local\Temp\tmpah4b5lyg.pyomo.lp:616: warning: lower bound of variable 'x137' redefined
C:\Users\Isabela\AppData\Local\Temp\tmpah4b5lyg.pyomo.lp:616: warning: upper bound of variable 'x137' redefined
89 rows, 73 columns, 261 non-zeros
5 integer variables, all of which are binary
621 lines were read
Writing problem data to 'C:\Users\Isabela\AppData\Local\Temp\tmpgyga0dd7.glpk.glp'...
520 lines were written
GLPK Integer Optimizer 5.0
89 rows, 73 columns, 261 non-zeros
5 integer variables, all of which are binary
Preprocessing...
2 constraint coefficient(s) were reduced
80 rows, 57 columns, 217 non-zeros
5 intege

{'Problem': [{'Name': 'unknown', 'Lower bound': 689.947938359017, 'Upper bound': 689.947938359017, 'Number of objectives': 1, 'Number of constraints': 89, 'Number of variables': 73, 'Number of nonzeros': 261, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '5', 'Number of created subproblems': '5'}}, 'Error rc': 0, 'Time': 0.02189922332763672}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [14]:
# Print results
# model.pprint()

# > Objective function
print(' ** Función objetivo: {0:0.3f}  **'.format(model.obj_rule.expr() ))

# > Binary Variable (distillation columns selected)
for c in model.y:
    if c == 1:
        print(' ** La columna {} está seleccionada'.format(c))


 ** Función objetivo: 689.948  **
